In [23]:
import kfp
import kfp.dsl as dsl
from kfp import components
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount
import pprint
import numpy as np


@dsl.pipeline(
    name="blueberry detection",
    description="blueberry detection pipeline"
)
def full_pipeline(size: str="2Gi"):
    
    vop = dsl.PipelineVolume(
        pvc="test-notebook-datavol-1",
        name="test-notebook-datavol-1",
    )
    
    download_component = kfp.components.load_component_from_file('download_model.yaml')
    test_component = kfp.components.load_component_from_file('test_component.yaml')
    postprocessing_model = kfp.components.load_component_from_file('postprocessing_model.yaml')
    
    #UNET3
    model_dir = "/mnt/model/"
    model_name = "blueberry_row_detection_model.pt"
    model_path = model_dir + model_name
    
    download_component_task = download_component(
        "biosens-test-inference-service", # minIO bucket where the data is stored
        "/mnt" # root folder in the notebook
        ).add_pvolumes({"/mnt": vop})
    download_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    test_component_task = test_component(
        model_path,
        "UnetPlusPlus", # specifying model type
        "cuda", # specifying the device for running inference service
        download_component_task.output
        ).add_pvolumes({"/mnt": vop})
    test_component_task.set_gpu_limit(1)
    test_component_task.execution_options.caching_strategy.max_cache_staleness = "P0D"
    
    postprocessing_model = postprocessing_model(
        "blueberry-results", # minIO bucket for uploading results
        "/mnt/DataTest/final_results_folder/unetpp_test_parcel_belanovica.tiff", # minIO path for uploading tif file that contains detected rows  
        "/mnt/DataTest/final_results_folder/unetpp_test_rgb_masked_with_red_belanovica.tiff", # minIO path for uploading tif file that contains detected rows overlaped on rgb orthomosaic 
        "/mnt/DataTest/final_results_folder/unetpp_test_parcel_belanovica.shp", # minIO path for uploading shp file that contains detected rows
        test_component_task.output).add_pvolumes({"/mnt": vop}) 
    
    
    print("gotovo sve")
if __name__ == '__main__':
    file_name = "full_pipeline.yaml"
    kfp.compiler.Compiler().compile(full_pipeline, file_name)

gotovo sve


In [24]:
import kfp
import random
client=kfp.Client()
filepath = '/home/jovyan/test-notebook-datavol-1/' + file_name
name = 'Blueberry_test_timestamp'
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

Uploaded pipeline: Blueberry_test_newUNetOnGPU16
